In [1]:
# It is based on LoRA paper: https://arxiv.org/abs/2106.09685

import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    IntervalStrategy,
)

from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
MODEL_NAME = "../models/Llama-2-7b-chat-hf/"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [4]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

model.modules()

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4194304 || all params: 6742609920 || trainables%: 0.06220594176090199


In [6]:
data = load_dataset("csv", data_files="../data/train_person_10.csv")

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['ExperiencedEmotion1', 'session', 'context', 'User'],
        num_rows: 10303
    })
})

In [8]:
data["train"][0]

{'ExperiencedEmotion1': '"SHAME_SHYNESS"',
 'session': 'VPN01',
 'context': 'There are 8 classes of non-verbal behaviour with sub-classes that you can use in your analysis. In brackets are the short explanations of the meaning of each behaviour:\n- Gaze: DOWN (looking down), DOWN_LEFT (looking down left), DOWN_RIGHT (looking down right), LEFT (looking left), RIGHT (looking right), STRAIGHT AT INTERVIEWER (looking straight at interviewer), UP (looking up), UP_LEFT (looking up left), UP_RIGHT (looking up right).\n- Eyes: PINCH (eyes are squeezed together), BLINK_REPEATEDLY (person blinks more than once in a row), CLOSE (eyes are closed), WIDEN (person widens eyes)\n- Ekman expression: FEAR (the person shows the typical characteristics of the Ekman expression fear), DISGUST (the person shows the typical characteristics of the Ekman expression disgust), JOY (the person shows the typical characteristics of the Ekman expression joy), SURPRISE (the person shows the typical characteristics of 

In [9]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<context>: {data_point["context"]}
<assistant>: {data_point["ExperiencedEmotion1"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [10]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/10303 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
data

Dataset({
    features: ['ExperiencedEmotion1', 'session', 'context', 'User', 'input_ids', 'attention_mask'],
    num_rows: 10303
})

In [12]:
import warnings

# Proper regular expression to match the warning message
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly")


In [13]:
training_args = transformers.TrainingArguments(
    output_dir="./saves",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    #save_strategy="epoch",
    save_strategy=IntervalStrategy.STEPS,  # change this line
    save_total_limit=1,  # add this line
    logging_strategy="epoch",
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2576,0.090300
5152,0.005300
7728,0.004400
10304,0.003800
12880,0.003600


TrainOutput(global_step=12880, training_loss=0.021477269089740257, metrics={'train_runtime': 62893.8108, 'train_samples_per_second': 0.819, 'train_steps_per_second': 0.205, 'total_flos': 3.748648936632238e+18, 'train_loss': 0.021477269089740257, 'epoch': 5.0})

In [14]:
model.save_pretrained("./models/llama-2-7b-chat-hf-llm-emo-person-10-finetuned-peft/")